In [1]:
import numpy as np 
import pandas as pd 
from sklearn.datasets import make_classification, make_circles
from sklearn.model_selection import train_test_split

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

#1. DNN 유방암데이터 

In [3]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# 유방암 data를 불러온 후, cancer 변수에 담아보자.
cancer = load_breast_cancer()

# 유방암 data를 학습용과 검증용 데이터 세트로 분리해보자.
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target,
                                                   test_size=0.3, shuffle = True)

In [4]:
X_train.shape, y_train.shape

((398, 30), (398,))

In [5]:
y_train = y_train.reshape(398,-1)

In [6]:
X_test.shape, y_test.shape

((171, 30), (171,))

In [7]:
y_test = y_test.reshape(171,-1)

In [8]:
X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train)

In [9]:
X_train.shape, y_train.shape

(torch.Size([398, 30]), torch.Size([398, 1]))

In [10]:
model = nn.Sequential(
    nn.Linear(30, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64,1),
    nn.Sigmoid()
)

In [11]:
model.train()

Sequential(
  (0): Linear(in_features=30, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=1, bias=True)
  (5): Sigmoid()
)

In [12]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(X_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/1000 Cost: 6.171877 Accuracy 37.19%
Epoch  100/1000 Cost: 0.199704 Accuracy 92.46%
Epoch  200/1000 Cost: 0.174031 Accuracy 93.22%
Epoch  300/1000 Cost: 0.150956 Accuracy 94.22%
Epoch  400/1000 Cost: 0.131451 Accuracy 95.23%
Epoch  500/1000 Cost: 0.113789 Accuracy 95.23%
Epoch  600/1000 Cost: 0.096434 Accuracy 96.48%
Epoch  700/1000 Cost: 0.082965 Accuracy 96.98%
Epoch  800/1000 Cost: 0.072231 Accuracy 97.24%
Epoch  900/1000 Cost: 0.063869 Accuracy 97.24%
Epoch 1000/1000 Cost: 0.058943 Accuracy 97.49%


In [13]:
model.eval()

Sequential(
  (0): Linear(in_features=30, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=1, bias=True)
  (5): Sigmoid()
)

In [14]:
with torch.no_grad(): # torch.no_grad()를 하면 gradient 계산을 수행하지 않는다.
    X_test = torch.FloatTensor(X_test)
    Y_test = torch.FloatTensor(y_test)
    hypothesis = model(X_test)
    prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
    correct_prediction = prediction.float() == Y_test # 실제값과 일치하는 경우만 True로 간주
    accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
    print('Accuracy:', accuracy * 100)

Accuracy: 95.90643274853801


#2. CNN fashion MNIST

In [6]:
import torch
from torchvision import datasets, transforms
import torch.nn.init
from torch.utils.data import Dataset

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [8]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [9]:
train_data = datasets.FashionMNIST(root='data',     # 다운로드 경로 지정
                                   train=True,        # 학습용 데이터셋 설정(True)
                                   download=True, 
                                   transform=transforms.ToTensor()   # 텐서로 변환       
                                  )

100%|██████████| 26421880/26421880 [00:00<00:00, 115835692.43it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 5936834.96it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 62301240.83it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 24125449.15it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [18]:
# test(학습용) 데이터셋 로드
test_data = datasets.FashionMNIST(root='data', 
                                  train=False,        # 검증용 데이터셋 설정(False)
                                  download=True, 
                                  transform=transforms.ToTensor()
                                 )

In [19]:
data_loader = torch.utils.data.DataLoader(dataset=train_data,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [20]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

In [21]:
# CNN 모델 정의
model = CNN().to(device)

In [22]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [23]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


In [24]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y느 ㄴ레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.468727201
[Epoch:    2] cost = 0.300220668
[Epoch:    3] cost = 0.262165844
[Epoch:    4] cost = 0.237292632
[Epoch:    5] cost = 0.216158241
[Epoch:    6] cost = 0.202416822
[Epoch:    7] cost = 0.184448361
[Epoch:    8] cost = 0.171463892
[Epoch:    9] cost = 0.158971936
[Epoch:   10] cost = 0.145281613
[Epoch:   11] cost = 0.135590851
[Epoch:   12] cost = 0.124171272
[Epoch:   13] cost = 0.116716482
[Epoch:   14] cost = 0.105538964
[Epoch:   15] cost = 0.0972194746


In [26]:
model.eval()

with torch.no_grad():
    X_test = test_data.test_data.view(len(test_data), 1, 28, 28).float().to(device)
    Y_test = test_data.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.84579998254776


#3. cifrar100 ResNET 이용하기